# Top2Vec Training with Pre-computed Embeddings (v1)

This notebook trains Top2Vec models for each subject (cs, math, physics) using all 6 pre-computed v1 embeddings, computes coherence scores (c_v), and saves results to CSV.

In [1]:
import os
import gc
import pandas as pd
import numpy as np
from pathlib import Path
from typing import List, Optional
from tqdm import tqdm
import warnings

from top2vec import Top2Vec
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from adapters import AutoAdapterModel
from transformers import AutoTokenizer
import torch

warnings.filterwarnings("ignore", category=FutureWarning)

## Configuration

In [2]:
VERSION = "v1"
LIST_SUBJECT = ["cs", "math", "physics"]

TRANSFORMERS = [
    "allenai/specter2",
    "sentence-transformers/all-MiniLM-L6-v2",
    "all-distilroberta-v1",
    "intfloat/e5-base-v2",
    "all-mpnet-base-v2",
    "BAAI/bge-base-en-v1.5"
]

EMBEDDING_DIMS = {
    "allenai/specter2": 768,
    "sentence-transformers/all-MiniLM-L6-v2": 384,
    "all-distilroberta-v1": 768,
    "intfloat/e5-base-v2": 768,
    "all-mpnet-base-v2": 768,
    "BAAI/bge-base-en-v1.5": 768
}

BASE_DIR = Path("../../dataset")
EMBEDDING_DIR = Path("../bertopic/embedding")
RESULT_DIR = Path("./")
MODEL_DIR = Path("./transformer")

RESULT_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)

## Helper Functions

In [3]:
def load_specter2():
    tokenizer = AutoTokenizer.from_pretrained("allenai/specter2_base")
    model = AutoAdapterModel.from_pretrained("allenai/specter2_base")
    model.load_adapter("allenai/specter2", source="hf", set_active=True)
    model.eval()
    if torch.cuda.is_available():
        model = model.cuda()
    return model, tokenizer


def encode_specter2(texts: List[str], model, tokenizer, batch_size: int = 32) -> np.ndarray:
    embeddings = []
    device = next(model.parameters()).device
    for i in tqdm(range(0, len(texts), batch_size), desc="  Generating"):
        batch = texts[i:i + batch_size]
        inputs = tokenizer(
            batch, padding=True, truncation=True,
            max_length=512, return_tensors="pt"
        ).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            batch_emb = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        embeddings.append(batch_emb)
    return np.vstack(embeddings)


def get_model_safe_name(model_name: str) -> str:
    return model_name.replace("/", "_").replace("-", "_")


def load_dataset(subject: str) -> Optional[pd.DataFrame]:
    file_path = BASE_DIR / subject / "emb" / f"{VERSION}.csv"
    if not file_path.exists():
        print(f"File not found: {file_path}")
        return None
    return pd.read_csv(file_path)


def load_mmap_embeddings(
    mmap_path: str,
    num_documents: int,
    embedding_dim: int,
    dtype: str = "float32"
) -> Optional[np.ndarray]:
    try:
        embs = np.array(np.memmap(
            mmap_path, dtype=dtype, mode="r",
            shape=(num_documents, embedding_dim)
        ))
        return normalize(embs)
    except FileNotFoundError:
        print(f"Embedding not found: {mmap_path}")
        return None
    except Exception as e:
        print(f"Error loading embeddings: {e}")
        return None


def train_top2vec_with_precomputed(
    documents: List[str],
    precomputed_embeddings: np.ndarray,
    transformer_name: str
) -> Top2Vec:
    num_docs = len(documents)

    if transformer_name == "allenai/specter2":
        st_model, alle_tokenizer = load_specter2()
    else:
        st_model = SentenceTransformer(transformer_name)

    original_embed_docs = Top2Vec._embed_documents

    def patched_embed_documents(self, train_corpus, batch_size):
        if len(train_corpus) == num_docs:
            print(f"  ✅ Using pre-computed document embeddings ({num_docs} docs)")
            return precomputed_embeddings
        else:
            print(f"  📝 Encoding vocab with {transformer_name} ({len(train_corpus)} words)")
            if transformer_name == "allenai/specter2":
                return encode_specter2(train_corpus, st_model, alle_tokenizer, batch_size=batch_size)
            return st_model.encode(train_corpus, batch_size=batch_size, show_progress_bar=False)

    Top2Vec._embed_documents = patched_embed_documents

    model = Top2Vec(
        documents=documents,
        embedding_model='all-MiniLM-L6-v2',
        contextual_top2vec=False,
        ngram_vocab=False,
    )

    Top2Vec._embed_documents = original_embed_docs
    del st_model

    return model


def calculate_coherence(
    model: Top2Vec,
    texts_tokenized: List[List[str]],
    dictionary: Dictionary,
    top_n: int = 5
) -> float:
    num_topics = model.get_num_topics()
    topic_words, _, _ = model.get_topics(num_topics)
    topic_words_sliced = topic_words[:, :top_n]

    cm = CoherenceModel(
        topics=topic_words_sliced.tolist(),
        texts=texts_tokenized,
        dictionary=dictionary,
        coherence='c_v',
        processes=1
    )

    return cm.get_coherence()

## Load Datasets & Tokenize

In [4]:
all_data = {}
all_texts_tokenized = {}
all_dictionaries = {}

for subject in LIST_SUBJECT:
    df = load_dataset(subject)
    if df is not None:
        all_data[subject] = df
        print(f"{subject}: {len(df):,} documents loaded")

        print(f"  Tokenizing for coherence...")
        texts_tokenized = [text.split() for text in tqdm(df['text'].fillna('').tolist(), desc=f"  {subject}")]
        all_texts_tokenized[subject] = texts_tokenized
        all_dictionaries[subject] = Dictionary(texts_tokenized)

print(f"\nTotal subjects loaded: {len(all_data)}")

cs: 165,756 documents loaded
  Tokenizing for coherence...


  cs: 100%|██████████| 165756/165756 [00:02<00:00, 72607.20it/s]


math: 157,085 documents loaded
  Tokenizing for coherence...


  math: 100%|██████████| 157085/157085 [00:01<00:00, 131652.46it/s]


physics: 146,311 documents loaded
  Tokenizing for coherence...


  physics: 100%|██████████| 146311/146311 [00:02<00:00, 63841.21it/s]



Total subjects loaded: 3


## Train Top2Vec for Each Subject × Embedding

In [5]:
results = []

for subject, df in all_data.items():
    print(f"\n{'='*70}")
    print(f"Subject: {subject.upper()} ({len(df):,} documents)")
    print(f"{'='*70}")

    documents = df['text'].fillna('').tolist()
    texts_tokenized = all_texts_tokenized[subject]
    dictionary = all_dictionaries[subject]

    for model_name in TRANSFORMERS:
        model_safe_name = get_model_safe_name(model_name)
        emb_dim = EMBEDDING_DIMS[model_name]

        embedding_path = str(EMBEDDING_DIR / subject / f"{model_safe_name}_{VERSION}.mmap")
        model_save_path = str(MODEL_DIR / subject / f"{model_safe_name}_{VERSION}")

        print(f"\n[{model_name}]")
        print(f"  Embedding dim: {emb_dim}")

        # Load existing model if available
        if os.path.exists(model_save_path):
            print(f"  Loading existing model from {model_save_path}")
            try:
                model = Top2Vec.load(model_save_path)
                n_topics = model.get_num_topics()
                print(f"  Topics: {n_topics}")

                print(f"  Calculating coherence...")
                coherence = calculate_coherence(model, texts_tokenized, dictionary)
                print(f"  Coherence (c_v): {coherence:.4f}")

                results.append({
                    "subject": subject,
                    "model": model_name,
                    "n_topics": n_topics,
                    "coherence": coherence
                })
                del model
                gc.collect()
                continue
            except Exception as e:
                print(f"  Failed to load, retraining: {e}")

        # Train new model
        embeddings = load_mmap_embeddings(embedding_path, len(documents), emb_dim)

        if embeddings is None:
            print(f"  ✗ Skipping (no embeddings)")
            results.append({
                "subject": subject,
                "model": model_name,
                "n_topics": None,
                "coherence": None,
                "error": "embeddings not found"
            })
            continue

        try:
            print(f"  Training Top2Vec...")
            model = train_top2vec_with_precomputed(documents, embeddings, model_name)

            n_topics = model.get_num_topics()
            print(f"  Topics found: {n_topics}")

            print(f"  Calculating coherence...")
            coherence = calculate_coherence(model, texts_tokenized, dictionary)
            print(f"  Coherence (c_v): {coherence:.4f}")

            # Save model
            model.save(model_save_path)
            print(f"  ✓ Saved to {model_save_path}")

            results.append({
                "subject": subject,
                "model": model_name,
                "n_topics": n_topics,
                "coherence": coherence
            })

            del embeddings, model
            gc.collect()

        except Exception as e:
            print(f"  ✗ Error: {e}")
            results.append({
                "subject": subject,
                "model": model_name,
                "n_topics": None,
                "coherence": None,
                "error": str(e)
            })

print("\n" + "="*70)
print("All training complete!")
print("="*70)


Subject: CS (165,756 documents)

[allenai/specter2]
  Embedding dim: 768
  Loading existing model from transformer/cs/allenai_specter2_v1
  Topics: 1
  Calculating coherence...
  Coherence (c_v): 0.2972

[sentence-transformers/all-MiniLM-L6-v2]
  Embedding dim: 384
  Loading existing model from transformer/cs/sentence_transformers_all_MiniLM_L6_v2_v1
  Topics: 808
  Calculating coherence...
  Coherence (c_v): 0.5718

[all-distilroberta-v1]
  Embedding dim: 768
  Loading existing model from transformer/cs/all_distilroberta_v1_v1
  Topics: 834
  Calculating coherence...
  Coherence (c_v): 0.4810

[intfloat/e5-base-v2]
  Embedding dim: 768
  Loading existing model from transformer/cs/intfloat_e5_base_v2_v1
  Topics: 1
  Calculating coherence...
  Coherence (c_v): 0.2440

[all-mpnet-base-v2]
  Embedding dim: 768
  Loading existing model from transformer/cs/all_mpnet_base_v2_v1
  Topics: 1031
  Calculating coherence...
  Coherence (c_v): 0.4928

[BAAI/bge-base-en-v1.5]
  Embedding dim: 768

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

There are adapters available but none are activated for the forward pass.
2026-02-14 18:26:58,039 - top2vec - INFO - Pre-processing documents for training
/home/nedo/Kuliah/TA/Program/.venv/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
2026-02-14 18:27:18,325 - top2vec - INFO - Downloading all-MiniLM-L6-v2 model
2026-02-14 18:27:22,391 - top2vec - INFO - Creating joint document/word embedding


  📝 Encoding vocab with allenai/specter2 (10779 words)


  Generating: 100%|██████████| 337/337 [00:04<00:00, 83.07it/s]
2026-02-14 18:27:26,460 - top2vec - INFO - Creating lower dimension embedding of documents


...

  📝 Encoding vocab with BAAI/bge-base-en-v1.5 (10779 words)


2026-02-14 18:35:36,777 - top2vec - INFO - Creating lower dimension embedding of documents


  ✅ Using pre-computed document embeddings (157085 docs)


2026-02-14 18:35:58,664 - top2vec - INFO - Finding dense areas of documents
2026-02-14 18:36:02,473 - top2vec - INFO - Finding topics


  Topics found: 31
  Calculating coherence...
  Coherence (c_v): 0.4369
  ✓ Saved to transformer/math/BAAI_bge_base_en_v1.5_v1

Subject: PHYSICS (146,311 documents)

[allenai/specter2]
  Embedding dim: 768
  Loading existing model from transformer/physics/allenai_specter2_v1
  Topics: 1
  Calculating coherence...
  Coherence (c_v): 0.3205

[sentence-transformers/all-MiniLM-L6-v2]
  Embedding dim: 384
  Loading existing model from transformer/physics/sentence_transformers_all_MiniLM_L6_v2_v1
  Topics: 687
  Calculating coherence...
  Coherence (c_v): 0.6354

[all-distilroberta-v1]
  Embedding dim: 768
  Loading existing model from transformer/physics/all_distilroberta_v1_v1
  Topics: 719
  Calculating coherence...
  Coherence (c_v): 0.4549

[intfloat/e5-base-v2]
  Embedding dim: 768
  Loading existing model from transformer/physics/intfloat_e5_base_v2_v1
  Topics: 1
  Calculating coherence...
  Coherence (c_v): 0.1187

[all-mpnet-base-v2]
  Embedding dim: 768
  Loading existing model fr

## Save & Display Results

In [6]:
results_df = pd.DataFrame(results)
csv_path = RESULT_DIR / f"coherence_results_{VERSION}.csv"
results_df.to_csv(csv_path, index=False)

print(f"\n{'='*70}")
print("COHERENCE SCORE SUMMARY")
print(f"{'='*70}\n")

if len(results_df) > 0 and results_df['coherence'].notna().any():
    valid = results_df.dropna(subset=['coherence'])
    pivot = valid.pivot(index='model', columns='subject', values='coherence')
    pivot['mean'] = pivot.mean(axis=1)
    pivot = pivot.sort_values('mean', ascending=False)
    print(pivot.round(4))

print(f"\nResults saved to: {csv_path}")


COHERENCE SCORE SUMMARY

subject                                     cs    math  physics    mean
model                                                                  
sentence-transformers/all-MiniLM-L6-v2  0.5718  0.5415   0.6354  0.5829
all-mpnet-base-v2                       0.4928  0.4805   0.5564  0.5099
all-distilroberta-v1                    0.4810  0.4664   0.4549  0.4674
BAAI/bge-base-en-v1.5                   0.4634  0.4369   0.4495  0.4499
allenai/specter2                        0.2972  0.5622   0.3205  0.3933
intfloat/e5-base-v2                     0.2440  0.3100   0.1187  0.2242

Results saved to: coherence_results_v1.csv


In [7]:
print("\nTop Models per Subject:")
print("-" * 110)

for subject in LIST_SUBJECT:
    subject_results = results_df[results_df['subject'] == subject]
    if len(subject_results) > 0 and subject_results['coherence'].notna().any():
        top_models = subject_results.nlargest(6, 'coherence')
        for rank, (_, row) in enumerate(top_models.iterrows(), 1):
            print(f"{subject.upper():10s} | #{rank} | {row['model']:45s} | {row['coherence']:.4f}")
        print("-" * 110)


Top Models per Subject:
--------------------------------------------------------------------------------------------------------------
CS         | #1 | sentence-transformers/all-MiniLM-L6-v2        | 0.5718
CS         | #2 | all-mpnet-base-v2                             | 0.4928
CS         | #3 | all-distilroberta-v1                          | 0.4810
CS         | #4 | BAAI/bge-base-en-v1.5                         | 0.4634
CS         | #5 | allenai/specter2                              | 0.2972
CS         | #6 | intfloat/e5-base-v2                           | 0.2440
--------------------------------------------------------------------------------------------------------------
MATH       | #1 | allenai/specter2                              | 0.5622
MATH       | #2 | sentence-transformers/all-MiniLM-L6-v2        | 0.5415
MATH       | #3 | all-mpnet-base-v2                             | 0.4805
MATH       | #4 | all-distilroberta-v1                          | 0.4664
MATH       | #5 | BAAI/